In [1]:
import h5py
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, find_peaks
import numpy as np
import datetime
import pandas as pd
import obspy
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import time
import glob
import os
import pytz
from tqdm import tqdm
import csv
import matplotlib.dates as mdates
import csv
import re
from templatematching import *




ModuleNotFoundError: No module named 'templatematching'

In [ ]:
#loading data to run

def get_file_list(base_path,start_date,end_date):
    file_list = []
    actual_date = start_date
    
    while actual_date <= end_date:
        #new_decimator is because is downsample data, usually is just decimator
        search_pattern = f"{base_path}/new_decimator_{actual_date}*"
        file_list.extend(glob.glob(search_pattern))
        actual_date = increase_date(actual_date)
        file_list.sort()
    return file_list
    
# defining the other fuction to increment the date
def increase_date(date_str):
    from datetime import datetime,timedelta
    date_format = "%Y-%m-%d"
    date = datetime.strptime(date_str,date_format)
    next_date = date + timedelta(days=1)
    return next_date.strftime(date_format)
    

In [ ]:
#testing fuctions
base_path = "/data/fast1/veronica-scratch-rainier-downsampling/drive1_ds"
start_date = "2023-08-25"
end_date = "2023-08-28"
file_list = get_file_list(base_path,start_date,end_date)
len(file_list)

In [ ]:
#block de prueba para ver como cambia la fecha de inicio
chan_min=0
chan_max=-1

#data_file = h5py.File('/data/data5/Converted/whidbey_2022-05-17_08-39-00_UTC_014654.h5','r')
data_file = h5py.File('/data/fast1/veronica-scratch-rainier-downsampling/drive1_ds/new_decimator_2023-08-25_00.00.00_UTC.h5','r')
#data_file = h5py.File('/data/fast0/dominik-scratch/Shared/rainier_earthquake_2024_02_07/decimator_2024-02-07_19.11.00_UTC.h5','r')

this_data0 = np.array(data_file['Acquisition/Raw[0]/RawData'][:,chan_min:chan_max])
this_time0 = np.array(data_file['Acquisition/Raw[0]/RawDataTime'])
            
attrs=dict(data_file['Acquisition'].attrs)

data_file.close()


In [ ]:
#block de prueba para ver como cambia la fecha de inicio
chan_min=0
chan_max=-1

#data_file = h5py.File('/data/data5/Converted/whidbey_2022-05-17_08-39-00_UTC_014654.h5','r')
data_file = h5py.File('/data/fast1/veronica-scratch-rainier-downsampling/drive1_ds/new_decimator_2023-08-31_23.59.00_UTC.h5','r')
#data_file = h5py.File('/data/fast0/dominik-scratch/Shared/rainier_earthquake_2024_02_07/decimator_2024-02-07_19.11.00_UTC.h5','r')

this_data1 = np.array(data_file['Acquisition/Raw[0]/RawData'][:,chan_min:chan_max])
this_time1 = np.array(data_file['Acquisition/Raw[0]/RawDataTime'])
            
attrs=dict(data_file['Acquisition'].attrs)

data_file.close()

In [ ]:
import datetime

# Marca de tiempo en PT (hora del Pacífico)
timestamp_pt0 = this_time0[0]
timestamp_pt1 = this_time1[0]

# Diferencia horaria entre PT y UTC en segundos
pt_offset_seconds = -25200  # Pacific Time (PT) is 8 hours behind UTC

# Convertir la marca de tiempo de PT a UTC
timestamp_utc0 = timestamp_pt0 + pt_offset_seconds
timestamp_utc1 = timestamp_pt1 + pt_offset_seconds

# Convertir a objeto datetime en PT y en UTC
datetime_pt0 = datetime.datetime.fromtimestamp(timestamp_pt0 / 1e6)
datetime_utc0 = datetime.datetime.utcfromtimestamp(timestamp_utc0 / 1e6)

datetime_pt1 = datetime.datetime.fromtimestamp(timestamp_pt1 / 1e6)
datetime_utc1 = datetime.datetime.utcfromtimestamp(timestamp_utc1 / 1e6)


#print("inicial Tiempo en PT:", datetime_pt0)
print("inicial Tiempo en UTC:", datetime_utc0)

#print("final Tiempo en PT:", datetime_pt1)
print("final Tiempo en UTC:", datetime_utc1)



In [ ]:
#loading templates

template_list = glob.glob('/data/data4/veronica-scratch-rainier/templates-files/*')


In [ ]:
# Base directory to save files
base_directory = '/data/data4/veronica-scratch-rainier/test_corr'
#the folder contain in the name_numbers date of the template used

#base_directory = '/home/velgueta/notebooks/RainierDas' #test

corrs = "corrs"
#temp = "informacion de la ultima parte de cada template"

daystart = file_list[0]
dayend = file_list[-1]

# Extract date, hour, and minute portion from file path, excluding "new" and "decimator"
daystart_parts = os.path.basename(daystart).split('_')
daystart_date_time = "_".join(daystart_parts[2:4])  # Extract the date and time parts, excluding "new_decimator" and "UTC.h5"

dayend_parts = os.path.basename(dayend).split('_')
dayend_date_time = "_".join(dayend_parts[2:4])  # Extract the date and time parts, excluding "new_decimator" and "UTC.h5"

chan_min = 1000
chan_max = 2500
channel_number = chan_max -chan_min
low_cut1 = 2
hi_cut1 = 9.0
#fs=attrs['MaximumFrequency']*2
fs = 20
samples_per_file = 60*fs
b, a = butter(2, (low_cut1, hi_cut1), 'bp', fs=fs)

# Buiding outputfiles and correlations for each template on the list

In [ ]:
start_time = time.perf_counter()

for i, file in tqdm(enumerate(file_list)):
    for j, tem in tqdm(enumerate(template_list)):
        try:
            # Load data file and template
            with h5py.File(file, "r") as f, h5py.File(tem, "r") as d:
                template = np.array(d['Acquisition/Raw[0]/RawData'][:, 0:-1])
                raw_data = np.array(f['Acquisition/Raw[0]/RawData'][:, chan_min:chan_max-1])
                timestamps = np.array(f['Acquisition/Raw[0]/RawDataTime'])
                
                # Filter data
                data_filt = filtfilt(b, a, raw_data, axis=0)
                
                # Compute correlations
                corrs = window_and_correlate(template, data_filt)
                corrs2 = corrs.reshape((int(samples_per_file), channel_number-1))
                corrs3 = np.sum(corrs2, axis=1) / channel_number
                
                # Create output folder name
                folder_name_parts = os.path.splitext(os.path.basename(tem))[0].split('_')[:2]
                folder_name = '_'.join(folder_name_parts)
                folder_output = os.path.join(base_directory, folder_name)
                
                # Create folder if it doesn't exist
                if not os.path.exists(folder_output):
                    os.mkdir(folder_output)
                
                # Save correlation results
                outfile_name = os.path.join(folder_output, f'corrs_{i}_.npy')
                np.save(outfile_name, corrs3)
        except Exception as e:
            print(f"Error processing file {file} with template {tem}: {e}. Skipping this file and moving to the next template.")
            continue

end_time = time.perf_counter()
execution_time = end_time - start_time
print(f"The code took {execution_time} seconds.")


In [ ]:
#time vector for plotting correlations

#time steps
#timesteps = len(file_list)* samples_per_file
# Extraer las fechas de inicio y fin del nombre de la carpeta de salida

#just formatting

# Convertir las cadenas de tiempo a objetos datetime
#start_datetime = pd.to_datetime(daystart_date_time, format="%Y-%m-%d_%H.%M.%S")
#end_datetime = pd.to_datetime(dayend_date_time, format="%Y-%m-%d_%H.%M.%S")

#final vector
#time_range = pd.date_range(start=start_datetime, end=end_datetime, periods=timesteps)
import datetime

# Suponiendo que tienes daystart_date_time y dayend_date_time definidos como cadenas de tiempo

# Calcular el número total de pasos de tiempo
timesteps = len(file_list) * samples_per_file

# Convertir las cadenas de tiempo a objetos datetime
start_datetime = datetime.datetime.strptime(daystart_date_time, "%Y-%m-%d_%H.%M.%S")
end_datetime = datetime.datetime.strptime(dayend_date_time, "%Y-%m-%d_%H.%M.%S")

# Crear el rango de fechas o tiempo
time_range = [start_datetime + i * (end_datetime - start_datetime) / (timesteps - 1) for i in range(timesteps)]



In [ ]:
#test para un tiempo mas preciso

# Marca de tiempo inicial y final en UTC
start_time_utc = datetime_utc0
end_time_utc = datetime_utc1

# Calcular el número total de pasos de tiempo
timesteps = (len(file_list)) * (samples_per_file)

# Crear el rango de fechas o tiempo
time_range = [start_time_utc + i * (end_time_utc - start_time_utc) / (timesteps - 1) for i in range(timesteps)]


In [ ]:
# Directorio base donde se encuentran las carpetas
base_directory = '/data/data4/veronica-scratch-rainier/test_corr'

# Obtener una lista de todas las carpetas en el directorio base
folders = [folder for folder in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, folder))]

# Inicializar una lista para almacenar los datos concatenados por cada carpeta
concatenated_data_per_folder = []

# Iterar sobre las carpetas y cargar los archivos .npy
for folder in folders:
    folder_path = os.path.join(base_directory, folder)
    npy_files = [np.load(os.path.join(folder_path, file)) for file in os.listdir(folder_path) if file.endswith('.npy')]
    concatenated_data_per_folder.append(np.concatenate(npy_files, axis=0))

# Crear el rango de fechas o tiempo con exactamente 20 objetos datetime por segundo
# Asumiendo que ya tienes definido time_range adecuadamente

# Graficar los datos para cada carpeta
fig, axs = plt.subplots(len(folders), 1, figsize=(10, 5*len(folders)))

for i, folder_data in enumerate(concatenated_data_per_folder):
    axs[i].plot(time_range, folder_data, label=f'Folder {folders[i]}', color='blue', linestyle='-')
    axs[i].set_title(f'Template {folders[i]}')
    axs[i].set_xlabel('Time')
    axs[i].set_ylabel('Correlation Value')
    axs[i].legend()  # Add legend to each subplot
    axs[i].set_ylim([0,0.5])  # Set y-axis limits from 0 to 0.5

plt.tight_layout()
plt.grid()
plt.show()

# add thresold and detections stuff

In [ ]:

# Define los umbrales
thresholds = [0.5, 0.1, 0.05, 0.04, 0.035, 0.03]

# Directorio donde se guardarán los archivos de texto
output_directory = "textfiles_results_thresholds"

# Itera sobre cada carpeta
for folder, folder_data in zip(folders, concatenated_data_per_folder):
    # Crear directorio si no existe
    folder_output_directory = os.path.join(output_directory, folder)
    if not os.path.exists(folder_output_directory):
        os.makedirs(folder_output_directory)
    
    # Ruta del archivo de salida
    output_file_path = os.path.join(folder_output_directory, f"detections_results_{folder}.txt")
    
    # Abre el archivo de salida en modo escritura
    with open(output_file_path, 'w') as file:
        # Escribe los encabezados
        file.write("Threshold\tNumber of Detections\tDetection Times (UTC)\n")

        # Itera sobre cada umbral
        for threshold in thresholds:
            # Encuentra los índices donde los valores superan el umbral
            indices_above_threshold = np.where(np.abs(folder_data) > threshold)[0]
            diff_indices = np.diff(indices_above_threshold)
            group_changes = np.where(diff_indices > 10)[0]
            detection_groups = np.split(indices_above_threshold, group_changes + 1)

            # Itera sobre cada grupo de detección
            for group in detection_groups:
                # Verifica si el grupo no está vacío
                if len(group) > 0:
                    # Toma solo la primera fecha en cada grupo
                    first_detection_time_utc = time_range[group[0]].strftime('%Y-%m-%d_%H.%M.%S')
                    #first_detection_time_utc = datetime.datetime.utcfromtimestamp(time_range[group[0]].timestamp()).strftime('%Y-%m-%d_%H.%M.%S')
                    
                    # Escribe los datos en una fila separados por tabulaciones
                    file.write(f"{threshold}\t{len(detection_groups)}\t{first_detection_time_utc}\n")
                else:
                    print("Grupo vacío encontrado para el umbral:", threshold)

# Mensaje de confirmación
print(f"Los resultados se han guardado en {output_directory}")
